In [1]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

list=([1,5,4],
    [1,5,None],
    [1,5,1],
    [1,5,4],
    [2,5,1],
    [2,5,2],
    [2,5,None],
    [2,5,None],
     [2,5,4])
df=spark.createDataFrame(list,['I_id','p_id','xyz'])
df.show()

+----+----+----+
I_id|p_id| xyz|
+----+----+----+
 1| 5| 4|
 1| 5|null|
 1| 5| 1|
 1| 5| 4|
 2| 5| 1|
 2| 5| 2|
 2| 5|null|
 2| 5|null|
 2| 5| 4|
+----+----+----+

In [2]:
w= Window().partitionBy("I_id","p_id").orderBy(F.col("xyz").asc_nulls_first())
w2= Window().partitionBy("I_id","p_id").orderBy(F.col("xyz").asc_nulls_first()).rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df.withColumn("xyz1",F.count(F.col("xyz").isNotNull()).over(w))\
.withColumn("xyz2", F.max(F.row_number().over(w)).over(w2))\
.withColumn("xyz3", F.first("xyz1").over(w))\
.withColumn("xyz4", F.when((F.col("xyz2")-F.col("xyz3"))%2!=0, (F.col("xyz2")-F.col("xyz3"))+1).otherwise(F.col("xyz2")-F.col("xyz3")))\
.withColumn("xyz4", (F.col("xyz4")/2).cast("int"))\
.withColumn("xyz6", F.col("xyz4")+F.col("xyz3"))\
.withColumn("xyz5", F.row_number().over(w))\
.withColumn("medianr", F.when(F.col("xyz6")==F.col("xyz5"), F.col("xyz")).otherwise(F.lit(None)))\
.withColumn("medianr2", (F.mean("medianr").over(w2)).cast("int"))\
.withColumn("xyz", F.when(F.col("xyz").isNull(), F.col("medianr2")).otherwise(F.col("xyz")))\
.drop("xyz1","xyz2","xyz3","xyz4","xyz5","xyz6","medianr","medianr2")\
.orderBy("I_id").show()

+----+----+---+
I_id|p_id|xyz|
+----+----+---+
 1| 5| 4|
 1| 5| 1|
 1| 5| 4|
 1| 5| 4|
 2| 5| 2|
 2| 5| 2|
 2| 5| 1|
 2| 5| 2|
 2| 5| 4|
+----+----+---+

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [4]:
list=[{(11,[1,2,3],[1.0,1.0])},
      {11,[1,2],[1.0,1.0]},
       {11,[1,2],[1.0,1.0]},
       {11,[1,4],[1.0,1.0]},
       {11,[1,3,4],[1.0,1.0]}]

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2595636507959732> in <module> 
 ----> 1 list=[{(11,[1,2,3],[1.0,1.0])},
 2 { 11 , [ 1 , 2 ] , [ 1.0 , 1.0 ] } , 
 3 { 11 , [ 1 , 2 ] , [ 1.0 , 1.0 ] } , 
 4 { 11 , [ 1 , 4 ] , [ 1.0 , 1.0 ] } , 
 5 {11,[1,3,4],[1.0,1.0]}]

 TypeError : unhashable type: 'list'

In [5]:
df=spark.createDataFrame(list, ['features_array'])
df.show(truncate=False)

+---------------------------+
features_array |
+---------------------------+
[11, [1, 2, 3], [1.0, 1.0]]|
[11, [1, 2], [1.0, 1.0]] |
[11, [1, 2], [1.0, 1.0]] |
[11, [1, 4], [1.0, 1.0]] |
[11, [1, 3, 4], [1.0, 1.0]]|
+---------------------------+

In [6]:
df = spark.createDataFrame([
    (0, "a"),
    (1, "a"),
    (1, "b"),
    (1, "c"),
    (2, "a"),
    (2, "b"),
    (2, "b"),
    (2, "b"),
    (2, "c"),
    (0, "a"),
    (1, "b"),
    (1, "b"),
    (2, "cc"),
    (3, "a"),
    (4, "a"),
    (5, "c")
], ["id", "category"])

In [7]:
pivoted = df.groupBy("id").pivot("category").count().na.fill(0)

In [8]:
from pyspark.ml.feature import VectorAssembler

input_cols = [x for x in pivoted.columns if x != id]

result = (VectorAssembler(inputCols=input_cols, outputCol="features")
    .transform(pivoted)
    .select("id", "features"))


df=result.withColumnRenamed("features", "features_array")
df.show(truncate=False)

+---+---------------------+
id |features_array |
+---+---------------------+
0 |(5,[1],[2.0]) |
5 |(5,[0,3],[5.0,1.0]) |
1 |[1.0,1.0,3.0,1.0,0.0]|
3 |(5,[0,1],[3.0,1.0]) |
2 |[2.0,1.0,3.0,1.0,1.0]|
4 |(5,[0,1],[4.0,1.0]) |
+---+---------------------+

In [9]:
df.withColumn("features1", F.split(F.regexp_replace((F.regexp_replace(F.regexp_replace(F.col("features_array").cast("string"),'\(','['),'\)',']')),'\[|]',''),',').cast(ArrayType(IntegerType()))).withColumn("sum_of_features", F.expr("""aggregate(features1, 0, (acc, x) -> acc + x)""")).show()


+---+--------------------+---------------+---------------+
 id| features_array| features1|sum_of_features|
+---+--------------------+---------------+---------------+
 0| (5,[1],[2.0])| [5, 1, 2]| 8|
 5| (5,[0,3],[5.0,1.0])|[5, 0, 3, 5, 1]| 14|
 1|[1.0,1.0,3.0,1.0,...|[1, 1, 3, 1, 0]| 6|
 3| (5,[0,1],[3.0,1.0])|[5, 0, 1, 3, 1]| 10|
 2|[2.0,1.0,3.0,1.0,...|[2, 1, 3, 1, 1]| 8|
 4| (5,[0,1],[4.0,1.0])|[5, 0, 1, 4, 1]| 11|
+---+--------------------+---------------+---------------+

In [10]:
df1.select(F.sum("sum_of_features")).show()

+--------------------+
sum(sum_of_features)|
+--------------------+
 57|
+--------------------+

In [11]:
result1.printSchema()

root
-- id: long (nullable = true)
-- features: vector (nullable = true)
-- features1: array (nullable = true)
 |-- element: long (containsNull = true)

In [12]:
%scala
val df=Seq(
       (1,-7.1732833,32.0414966),
       (2,-7.1732844,32.0414406),
       (3,-7.1732833,32.0414966),
       (4,-7.1732833,32.0414966),
       (5,-7.1732833,32.0414966),
       (6,-7.1732833,32.0414966)
        ).toDF("seq","longitude","latitude")
df.show()

+---+----------+----------+
seq| longitude| latitude|
+---+----------+----------+
 1|-7.1732833|32.0414966|
 2|-7.1732844|32.0414406|
 3|-7.1732833|32.0414966|
 4|-7.1732833|32.0414966|
 5|-7.1732833|32.0414966|
 6|-7.1732833|32.0414966|
+---+----------+----------+

df: org.apache.spark.sql.DataFrame = [seq: int, longitude: double ... 1 more field]

In [13]:
%scala

df= spark.createDataFrame(list,("seq","longitude", "latitude"))
df.show()

In [14]:
%scala
val df=Seq(
       (1,-7.1732833,32.0414966),
       (2,-7.1732844,32.0414406),
       (3,-7.1732833,32.0414966),
       (4,-7.1732833,32.0414966),
       (5,-7.1732833,32.0414966),
       (6,-7.1732833,32.0414966)
        ).toDF("seq","longitude","latitude")


import org.apache.spark.sql.functions.lead 
import org.apache.spark.sql.functions.col 

val w = org.apache.spark.sql.expressions.Window.orderBy("date").orderBy("seq")

df.withColumn("destination_longitude", lead("longitude",1,0).over(w)).withColumn("destination_latitude", lead("latitude",1,0).over(w)).select(col("longitude").alias("origin_longitude"),col("destination_longitude"),col("latitude").alias("origin_latitude"),col("destination_latitude")).filter(col("destination_longitude")!==0.0).show()

+----------------+---------------------+---------------+--------------------+
origin_longitude|destination_longitude|origin_latitude|destination_latitude|
+----------------+---------------------+---------------+--------------------+
 -7.1732833| -7.1732844| 32.0414966| 32.0414406|
 -7.1732844| -7.1732833| 32.0414406| 32.0414966|
 -7.1732833| -7.1732833| 32.0414966| 32.0414966|
 -7.1732833| -7.1732833| 32.0414966| 32.0414966|
 -7.1732833| -7.1732833| 32.0414966| 32.0414966|
+----------------+---------------------+---------------+--------------------+

command-482160419578958:16: warning: method !== in class Column is deprecated: !== does not have the same precedence as ===, use =!= instead
df.withColumn("destination_longitude", lead("longitude",1,0).over(w)).withColumn("destination_latitude", lead("latitude",1,0).over(w)).select(col("longitude").alias("origin_longitude"),col("destination_longitude"),col("latitude").alias("origin_latitude"),col("destination_latitude")).filter(col("destination_longitude")!==0.0).show()
 ^
df: org.apache.spark.sql.DataFrame = [seq: int, longitude: double ... 1 more field]
import org.apache.spark.sql.functions.lead
import org.apache.spark.sql.functions.col
w: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6f68f14e

In [15]:
%python
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.types import *
cSchema = StructType([StructField("col", LongType())])
vals = [[0] for i in range(20)]
test_df = spark.createDataFrame(vals,schema=cSchema)

test_df.show(20)

+---+
col|
+---+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
+---+

In [16]:
test_df = test_df.replace(float('nan'), None)
test_df.show()

+----+
 col|
+----+
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
+----+

In [17]:
df = sqlContext.createDataFrame([
    ('www.example.com/researc', 'Research Reports'),
    ('www.example.com/careers', 'Careers'),
    ('www.example.com/blogs', 'blogs'),
    ('www.example.com', None),
    ('www.example.com/navigation', None),
    ('www.example.com', None),
    ('www.example.jp', None),
    ('', 'blogs')], ['A', 'B'])

df.show()

filter_mask = df.where(df['B'].isNotNull())
print("\n\nFilter Mask")
filter_mask.show()

print('\n\nfilter_mask[A]')
#filter_mask.select('A').show()

# Why is "response" returning everything?!
response = df.filter(filter_mask.A.isin(df.A))
print("\n\nResulting DF")
response.show()


+--------------------+----------------+
 A| B|
+--------------------+----------------+
www.example.com/r...|Research Reports|
www.example.com/c...| Careers|
www.example.com/b...| blogs|
 www.example.com| null|
www.example.com/n...| null|
 www.example.com| null|
 www.example.jp| null|
 | blogs|
+--------------------+----------------+



Filter Mask
+--------------------+----------------+
 A| B|
+--------------------+----------------+
www.example.com/r...|Research Reports|
www.example.com/c...| Careers|
www.example.com/b...| blogs|
 | blogs|
+--------------------+----------------+



filter_mask[A]


Resulting DF
+--------------------+----------------+
 A| B|
+--------------------+----------------+
www.example.com/r...|Research Reports|
www.example.com/c...| Careers|
www.example.com/b...| blogs|
 www.example.com| null|
www.example.com/n...| null|
 www.example.com| null|
 www.example.jp| null|
 | blogs|
+--------------------+----------------+

In [18]:
list=[['1'],['-2'],['-1'],['0']]
df=spark.createDataFrame(list, ["number"])
df.show()

+------+
number|
+------+
 1|
 -2|
 -1|
 0|
+------+

In [19]:
df.withColumn("number", F.abs(F.col("number")).cast('int')).show()

+------+
number|
+------+
 1|
 2|
 1|
 0|
+------+

In [20]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
list=[['USDINRFUTCUR23Feb201700000000FF00000000000001990067895000000000NNN*12'],
      ['USDINRFUTCUR24Feb201700000000FF00000000000001990067895000000000NNN*12'],
      ['USDINRFUTCUR25Feb201700000000FF00000000000001990067895000000000NNN*12']]

df=spark.createDataFrame(list,['col1'])

df.show(truncate=False)

df.withColumn("Currency1", F.col("col1").substr(0,3))\
  .withColumn("Currency2", F.col("col1").substr(4,3))\
  .withColumn("Type", F.col("col1").substr(7,6))\
  .withColumn("Time", F.expr("""substr(col1,13,length(col1))"""))\
  .drop("col1").show(truncate=False)

+---------------------------------------------------------------------+
col1 |
+---------------------------------------------------------------------+
USDINRFUTCUR23Feb201700000000FF00000000000001990067895000000000NNN*12|
USDINRFUTCUR24Feb201700000000FF00000000000001990067895000000000NNN*12|
USDINRFUTCUR25Feb201700000000FF00000000000001990067895000000000NNN*12|
+---------------------------------------------------------------------+

+---------+---------+------+---------------------------------------------------------+
Currency1|Currency2|Type |Time |
+---------+---------+------+---------------------------------------------------------+
USD |INR |FUTCUR|23Feb201700000000FF00000000000001990067895000000000NNN*12|
USD |INR |FUTCUR|24Feb201700000000FF00000000000001990067895000000000NNN*12|
USD |INR |FUTCUR|25Feb201700000000FF00000000000001990067895000000000NNN*12|
+---------+---------+------+---------------------------------------------------------+